In [1]:
import gensim.downloader as api
import gensim
from bs4 import BeautifulSoup
import os

In [2]:
ft = api.load('fasttext-wiki-news-subwords-300')

In [5]:
# Processing multiple HTML files in a folder
folder_path = r"../Week_2/wikipedia_talk_pages"
documents: list[str] = []
for filename in os.listdir(folder_path):
    if filename.endswith(".html"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f.read(), "html.parser")
            result = []

            # page title 
            title = soup.find("span", class_="mw-page-title-main")
            if title:
                result.append(title.get_text(strip=True)) # strip removes leading/trailing whitespace

            # main content
            root = soup.find("div", id="mw-content-text") # we create root to be sure we are in the content section
            content = root.find("div", class_="mw-parser-output") if root else None # it verifies root is not None
            # in content, we look for h2, h3, and p tags that are direct children (not nested deeper)

            if content:
                for el in content.find_all(["h2", "h3", "p"], recursive=False): # recursive=False ensures we only get direct children
                    text = el.get_text(" ", strip=True)
                    if text:
                        result.append(text)

            full_text = "\n\n".join(result) # every wiki page is stored as a single string with double newlines between sections
            if full_text:   # only add non-empty documents    
                documents.append(full_text)

print(f'The number of documents in the list is: {len(documents)}', "Type of documents:", type(documents[0]))
print("First 500 characters of the first document:\n", documents[0][:500]) 

The number of documents in the list is: 1225 Type of documents: <class 'str'>
First 500 characters of the first document:
 Glutamic acid

If high levels of glutamic acid are unhealthy, what would be considered a high level?

"Glutamic acid" and "glutamate" are often used interchangeably but, technically, glutamate is the anionic form of glutamic acid. This article doesn't recognize that. RJII 17:01, 9 September 2005 (UTC) [ reply ]

Response to above: whether it exists in the anionic or neutral form is going to be entirely dependent upon the pH - review the Henderson-Hasselbach equation. Cajolingwilhelm 01:30, 14 Fe


In [ ]:
# doc_vectors = gensim.models.keyedvectors.KeyedVectors(ft.vector_size, count=len(documents))
# for i, line in enumerate(documents):
#     # gensim provides procedures for preprocessing and stopword removal
#     text_without_stopwords = gensim.parsing.preprocessing.remove_stopwords(line)
#     tokens = gensim.utils.simple_preprocess(text_without_stopwords)
#     # the function get_mean_vector computes the average vector for all tokens
#     dv = ft.get_mean_vector(tokens)
#     doc_vectors.add_vector(i, dv)

In [14]:
import sys
# Evil hack from Week 2
sys.path.append("..")
import Search_Algorithms.semantic
import importlib
importlib.reload(Search_Algorithms.semantic)

engine = Search_Algorithms.semantic.SemanticSearchEngine(ft, documents)

while True:
    query = input("Hi! What do you want to search for?").lower().strip()
    if query == "":
        break
    r = engine.search(query, topn=10)
    print(f"Found {len(r)} results")
    for i, d in enumerate(r):
        print(f"#{i}: {d[:100]}")

Found 10 results
#0: Dog-like cat
#1: Cats in New Zealand
#2: Cat bite

This article appears to be quite negative about the subject, some of my feline friends use
#3: Geoffroy's cat
#4: Foldex cat
#5: Animal testing on cats
#6: Working cat

I don't really understand the correlation between feral cats and working cats. I don't 
#7: Chantilly-Tiffany

My parents live in Phoenix, Arizona and own one of these cats. It is a pretty lar
#8: Farm cat

This article is useful as a portrayal of the 'working cat' in agriculture. There are a who
#9: Cat breeds
